<a href="https://colab.research.google.com/github/molabokchi/bokchi_open_lab/blob/main/hpo_concept.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1.deepdriver 및 optuna 설치

In [2]:
pip install deepdriver

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.0/47.0 KB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 52.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.8/409.8 KB 37.4 MB/s eta 0:00:00
  Created wheel for assertpy: filename=assertpy-1.1-py3-none-any.whl size=42917 sha256=929719e0a1478fbf6fbdd138a2e5b0cf95e1f9fcb0f9bda1e4a6e9f86f491ec2
  Stored in directory: /root/.cache/pip/wheels/57/86/c9/1310be6ddfb540daa0bf1ac204526837aa0a8b0e79f32855ff
Successfully built assertpy
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.19.6
    Uninstalling protobuf-3.19.6:
      Successfully uninstalled protobuf-3.19.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency confli

In [3]:
pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.5/348.5 KB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.0/81.0 KB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.4/210.4 KB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 KB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 KB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.1/147.1 KB 15.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.6/112.6 KB 11.7 MB/s eta 0:00:00
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11136 sha256=fa844a9918d1329add86df3bde8339328d266f23e21a78492c71dd51e5dddc35
  Stored in directory: /root/.cache/pip/wheels/7f/1a/65/84ff8c386bec21fca6d220ea1f5498a0367883a78dd5ba6122
Successfully built pyperclip
  At

 # 2. deepdriver import

In [4]:
import deepdriver

# 3. 추후 deepdriver에 반영될 소스 

In [5]:
import optuna
from urllib.parse import quote_plus
from optuna.exceptions import DuplicatedStudyError

def create_hpo(exp_name):
    try:
        #to-do: /api/client/experiment/create/hpo 호출
        optuna.create_study(study_name=exp_name,
                            storage="postgresql://bokchi1:%s@ce-postg.bokchi.com:5432/hpo" % quote_plus("eiljbagADSfdsiles!@#gd3fds324234dgfsdg"),
                            direction='minimize')

    except DuplicatedStudyError:
        print("aleady exist !")
    else:
        print("please check url")

def objective(trial, exp_name, func, config):
    deepdriver.init(exp_name= exp_name)
    get_config(trial,config)
    y= func()
    deepdriver.log({"x":deepdriver.config.x,"y":y})
    deepdriver.finish()

    return y

def get_config(trial, config):
    #to-do config dict의 내용을 조건에  따라 하기와 같은 코드로 변환
    deepdriver.config.x = trial.suggest_float("x", -10, 10)

def get_hpo(exp_name):
    #to-do : 조건 서버에서 가져와서 파싱하기 /api/client/experiment/get/hpo
    config = [ {"key": "x" , "value" : {"range":[0.01,0.001]} }]
    return config

def get_wrapped_func(exp_name, func, config):
    return lambda trial: objective(trial, exp_name, func, config)

def load_hpo(exp_name):
    study = optuna.load_study(
        study_name=exp_name,
        storage="postgresql://bokchi1:%s@ce-postg.bokchi.com:5432/hpo" % quote_plus(
            "eiljbagADSfdsiles!@#gd3fds324234dgfsdg"))
    return study

def run_optimize(study,exp_name,func,config, count):
    study.optimize(get_wrapped_func(exp_name, func, config), n_trials=count)

def run_hpo( exp_name, func, count):
    config = get_hpo(exp_name)
    study = load_hpo(exp_name)
    run_optimize(study, exp_name,func,config, count)

# 4. 딥드라이버 셋팅 & 로그인 & hpo 생성 및 실행 

In [ ]:
exp_name = "exp_hpo8"
deepdriver.setting(http_host="54.180.86.146:9011", grpc_host="54.180.86.146:19051")
deepdriver.login(key="ZmIyNWQxNGJkMzUxYTVjODQ2NjM5NTgzOTM0YTM2OGE2ZmJiY2M2MWMwOWQ0OWFkNjU2YzNkM2UxMjA0YTVkZg==")

create_hpo(exp_name) #추후 deepdriver.create_hpo로


def func():
    x = deepdriver.config.x
    y = (x - 2) ** 2
    return y

run_hpo(exp_name, func, 50) #추후 deepdriver.run_hpo로

[I 2023-01-04 09:01:46,316] A new study created in RDB with name: exp_hpo8


please check url
2023-01-04 09:02:01,059 INFO [deepdriver] [experiment.py:25] - DeepDriver initialized
Team Name=molamola.python
Exp Name=exp_hpo8
Run Name=run-1
Run URL=http://54.180.86.146:9111/experi/molamola.python/exp_hpo8/run-1/run/chart


INFO:deepdriver:DeepDriver initialized
Team Name=molamola.python
Exp Name=exp_hpo8
Run Name=run-1
Run URL=http://54.180.86.146:9111/experi/molamola.python/exp_hpo8/run-1/run/chart
[I 2023-01-04 09:02:09,965] Trial 0 finished with value: 36.83275537395124 and parameters: {'x': -4.068999536492917}. Best is trial 0 with value: 36.83275537395124.


2023-01-04 09:02:13,043 INFO [deepdriver] [experiment.py:25] - DeepDriver initialized
Team Name=molamola.python
Exp Name=exp_hpo8
Run Name=run-2
Run URL=http://54.180.86.146:9111/experi/molamola.python/exp_hpo8/run-2/run/chart


INFO:deepdriver:DeepDriver initialized
Team Name=molamola.python
Exp Name=exp_hpo8
Run Name=run-2
Run URL=http://54.180.86.146:9111/experi/molamola.python/exp_hpo8/run-2/run/chart
[I 2023-01-04 09:02:21,041] Trial 1 finished with value: 62.53911162742245 and parameters: {'x': 9.908167400063206}. Best is trial 0 with value: 36.83275537395124.


...

INFO:deepdriver:DeepDriver initialized
Team Name=molamola.python
Exp Name=exp_hpo8
Run Name=run-12
Run URL=http://54.180.86.146:9111/experi/molamola.python/exp_hpo8/run-12/run/chart
[I 2023-01-04 09:04:12,017] Trial 11 finished with value: 0.05896072435360541 and parameters: {'x': 2.2428182949318387}. Best is trial 11 with value: 0.05896072435360541.


2023-01-04 09:04:15,121 INFO [deepdriver] [experiment.py:25] - DeepDriver initialized
Team Name=molamola.python
Exp Name=exp_hpo8
Run Name=run-13
Run URL=http://54.180.86.146:9111/experi/molamola.python/exp_hpo8/run-13/run/chart


INFO:deepdriver:DeepDriver initialized
Team Name=molamola.python
Exp Name=exp_hpo8
Run Name=run-13
Run URL=http://54.180.86.146:9111/experi/molamola.python/exp_hpo8/run-13/run/chart
[I 2023-01-04 09:04:23,133] Trial 12 finished with value: 1.6390535307483927 and parameters: {'x': 3.2802552599963777}. Best is trial 11 with value: 0.05896072435360541.


2023-01-04 09:04:26,194 INFO [deepdriver] [experiment.py:25] - DeepDriver initialized
Team Name=molamola.python
Exp Name=exp_hpo8
Run Name=run-14
Run URL=http://54.180.86.146:9111/experi/molamola.python/exp_hpo8/run-14/run/chart


INFO:deepdriver:DeepDriver initialized
Team Name=molamola.python
Exp Name=exp_hpo8
Run Name=run-14
Run URL=http://54.180.86.146:9111/experi/molamola.python/exp_hpo8/run-14/run/chart
